In [1]:
'''Función que nos genera una población inicial de 2k individuos y 
   recibe como parámetros: La 'k' y el hipercubo al que pertenecen 
   los vectores (o sea, la dimensión 'n' y el intervalo [a,b] al que
   pertenecen cada una de las entradas de los vectores).'''

import numpy as np

def Generar_Poblacion(k, n, intervalo):
    
   a, b = intervalo

   # Se genera una matriz de tamaño (2k, n) con valores aleatorios
   # uniformes (o sea, una población inicial de 2k individuos donde 
   # cada uno vive en Rn):
   poblacionMatriz = np.random.uniform(a,b, size=(2*k, n))

   # Para facilitarnos la vida, convertimos a la matriz en una lista
   # de listas:
   poblacionLista = poblacionMatriz.tolist()

   return poblacionLista


In [2]:
'''Definimos una función de aptitud que recibe como parámetros una
   función de prueba y una población de 2k individuos y retorna una
   lista con las probabilidades de selección (p_i) de todos los 
   individuos de la población. Además de que nos da al mejor individuo
   en la población.'''

def Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion):
    # Generamos una lista con las evaluaciones en la función objetivo
    evaluaciones = [funcion(individuo) for individuo in poblacion]

    #print("Evaluaciones:", [float(eval) for eval in evaluaciones], "\n")

    # Generamos una lista de las aptitudes de los individuos en la 
    # población.
    aptitudes = [np.exp(1e-9*(-evaluacion)) for evaluacion in evaluaciones]

    #print("Aptitudes:", [float(apt) for apt in aptitudes], "\n")

    # Encontramos el índice del mejor individuo (menor evaluación)
    mejorIndice = np.argmax(aptitudes)
    masApto = poblacion[mejorIndice]

    #print("El individuo más apto fue:", masApto, "\n")

    # Obtenemos el 'tamaño del pastel':
    sumaAptitudes = sum(aptitudes)

    # Calculamos las probabilidades de selección (p_i) para todos los
    # individuos:
    probabilidades = [(aptitud/sumaAptitudes) for aptitud in aptitudes]

    #print("Probabilidades de selección:", [float(pi) for pi in probabilidades])


    return probabilidades, masApto


**Cambié esto:**

In [3]:
'''Definimos una función que recibe la lista de probabilidades 
   de selección y nos retorna la que corresponde a la de las 
   probabilidades acumulativas.'''

def Probas_Acumulativas(probabilidades):

    # Calculamos las probabilidades acumulativas (q_i) para todos los
    # individuos:
    probasAcumulativas = np.cumsum(probabilidades)

    return probasAcumulativas # Retornamos las `probasAcumulativas` como
                              # un arreglo de NumPy (igual es iterable).


In [4]:
'''Estas funciones son las que nos ayudan a generar a los padres mediante
   el uso de la ruleta.'''
   

'''La primera es para seleccionar padres (uno) y recibe como parámetro 
   la lista de las probabilidades acumulativas.
   
   Nos retornará el índice en la lista que le corresponde al padre 
   elegido con esa regla.'''

import random
import math

def Seleccionar_Padre_Ruleta(probasAcumuladas):
   r = random.random() # Generamos un número aleatorio entre 0 y 1

   tol=1e-9
   for i, probAcumulada in enumerate(probasAcumuladas):
      if r<= probAcumulada or math.isclose(r, probAcumulada, abs_tol=tol):
         return i # Índice del individuo seleccionado.


'''Le segunda es para generar la lista de padres elegidos para 
   reproducirse. Necesitamos 2k nuevos individuos, así que se van a 
   elegir k parejas (cada par de padres produce dos hijos).
   La función recibe como parámetros: la lista de individuos, las
   probabilidades acumulativas y un máximo de intentos para intentar evitar
   la generación de clones (en este caso, vamos a considerar un máximo
   de 100 para cada elección de un segundo padre, pero tenemos la 
   apertura de modificarlo después).'''

def Generar_Parejas(individuos, probasAcumuladas, maxIntentos=100):
    parejas = []
    clonesGenerados = 0  # Contador de clones generados
    numIndividuos = len(individuos)

    if numIndividuos < 2:
        print("Error: No hay suficientes individuos para generar parejas.")
        return parejas, clonesGenerados

    # Como len(individuos)=2k, le pedimos que elija a len(individuos)/2
    # parejas de padres (que nos generarán 2k hijos).
    parejasNecesarias = int(numIndividuos/2)
    #print(f"Se necesitan {parejasNecesarias} parejas de padres.")

    for _ in range(parejasNecesarias):
        padre1_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

        intentos = 0
        while True:
            padre2_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

            if padre1_idx != padre2_idx:
                break  # Padres diferentes, salir del ciclo
            
            intentos += 1
            if intentos >= maxIntentos:  # Después de varios intentos, permitir clones
                padre2_idx = padre1_idx
                clonesGenerados += 2 # Cada pareja produce dos hijos
                #print(f"Se ha generado un clon tras {intentos} intentos.")
                break  # Salir del ciclo permitiendo el clon

        # Agregamos la pareja solo si hay padres válidos
        if padre1_idx is not None and padre2_idx is not None:
            parejas.append((individuos[padre1_idx], individuos[padre2_idx]))
        else:
            print("Error: No se pudieron seleccionar padres válidos.")

    return parejas, clonesGenerados


Las siguientes son las funciones necesarias para codificar un vector en $\mathbb{R}^n$ en arreglos de bits (la recuperamos de una tarea anterior):

**Cambié esto:**

In [5]:
'''Funciones auxiliares para la codificación en binario.'''

def codifica_real(x, n_bit, intervalo):
    """
    Codifica un número real en el intervalo [a,b] utilizando nBit bits y una partición uniforme en [a, b].

    Parámetros:
    x: número a real a codificar.
    nBit: número de bits a utilizar.
    a: Extremo izquierdo del intervalo.
    b: Extremo derecho del intervalo.

    Retorno:
    Arreglo binario que representa al número real x con nBit bits en el intervalo [a, b].
    """
    
    a, b = intervalo
    
    # Calcula la precisión de la representación.
    precision = (b - a) / ((2 ** n_bit) - 1)

    # Asegura que el número esté dentro del rango de la partición.
    x = max(a, min(b, x))

    # Calcula el índice del número en la partición.
    index = int((x - a) / precision)

    # Codifica el índice a binario usando nuestro codigo para codificar naturales.
    if index < 0 or index >= (1 << n_bit):
        raise ValueError(f"Índice fuera del rango representable con {n_bit} bits.")

    x_binario = [0] * n_bit
    for i in range(n_bit - 1, -1, -1):
        x_binario[i] = index & 1
        index >>= 1

    # Devuelve el arreglo binario que representa al número real 'x'.
    return x_binario

def codifica_vector(vector_reales, n_bit, intervalo):
    """
    Codifica un vector de números reales en un vector de vectores binarios utilizando nBit bits.

    Parámetros:
    vector_reales: Arreglo de números reales a codificar
    dim_x: Dimensión del vector vector_reales
    nBit: Número de bits a utilizar para las entradas de nuestro arreglo.
    a: Extremo izquierdo del intervalo.
    b: Extremo derecho del intervalo.

    Retorno:
    Arreglo de arreglos arreglos binarios, donde cada sub-arreglo representa un número real codificado en binario
    """
    
    dim_x = len(vector_reales)

    # Inicializa un arreglo vacío para almacenar los vectores binarios.
    vector_binario = []

    vector_binario = [codifica_real(vector_reales[i], n_bit, intervalo) 
                      for i in range(dim_x)]

    # Devuelve un arreglo de arreglos binarios.
    return vector_binario


#print(f"El vector {[1.4,1.2,3,1.5674]} en binario con {10} bits es:\n")
#print(codifica_vector([1.4,1.2,3,1.5674],10,(1,5)))


**Cambié esto:**

In [6]:
'''Codificación de los padres en vectores binarios del estilo
   [[0,1,1,1,...], ..., [1, 1, 1, 0,...]]'''

def Padres_Binarios(parejas, nBits, intervalo):
   parejasBinarias = [(codifica_vector(padre1, nBits, intervalo),
                       codifica_vector(padre2, nBits, intervalo))
                       for padre1, padre2 in parejas]
   
   return parejasBinarias


**Creo que esta se puede modificar:**

In [7]:
'''Función que realiza la cruza de n puntos. Recibe como parámetros: la
   lista de los padres (en arreglos binarios), el número de cortes que 
   se quieren hacer por cromosoma y el número de bits usados en la 
   codificación binaria. Retorna una lista de 2k hijos (como arreglos 
   binarios).'''

def Cruzar_N_Puntos(padresBinarios, nCortes, nBits):
    
    hijos = [] # Lista donde se guardará a los 2k hijos
    
    # Los `nCortes` son un parámetro recibido e idealmente es el que se
    # usará, en caso de que `nCortes` sea demasido grande, se hará el 
    # mayor número de cortes posible, o sea `nBits - 1`
    n = min(nCortes, (nBits - 1))
    
    for padre1, padre2 in padresBinarios:
        
        # Los hijos se van a crear cruzando cada cromosoma de los padres. 
        # Recordemos que cada pareja produce dos hijos.
        hijo1 = []
        hijo2 = []
        
        # Cruzamos cada cromosoma individualmente
        for cromo1, cromo2 in zip(padre1, padre2): # `zip(padre1, padre2)` toma los cromosomas 
                                                   # correspondientes de ambos padres al mismo 
                                                   # tiempo, es decir, empareja los cromosomas 
                                                   # que están en las mismas posiciones de las 
                                                   # listas `padre1` y `padre2`.
            
            # Determinar los puntos de cruce
            puntosCruce = sorted(random.sample(range(1, len(cromo1)), n))

            #print(puntosCruce) # En la primera iteración, si tenemos [1, 5]
                               # esto quiere decir que en el primer cromosoma
                               # de la pareja de padres, se van a hacer cruces
                               # en tres pedazos determinados por los dos cortes
                               # en las posiciones 1 y 5.
            
            # Inicializamos cromosomas vacíos para los hijos
            hijo1Cromo = []
            hijo2Cromo = []
            
            ultimoPunto = 0 # Punto en el que vamos para los cruces, inicia en 0.
            
            switch = False  # Esta bandera indica si debemos cambiar de padre
            
            # Alternamos entre segmentos de los padres en los puntos de cruza
            for punto in (puntosCruce + [len(cromo1)]): # Añadimos el valor len(cromo1) 
                                                        # al final de la lista de puntos de 
                                                        # cruza para asegurarnos de que el 
                                                        # último segmento después del último 
                                                        # punto de corte también se incluya. 
                                                        # De esta manera, recorremos todo el 
                                                        # cromosoma.
                
                # Seleccionamos los segmentos entre el último punto y el punto actual
                if switch:
                    hijo1Cromo += cromo2[ultimoPunto:punto] # El cromosoma del hijo1 recibe 
                                                            # parte del cromosoma del segundo
                                                            # padre.
                    hijo2Cromo += cromo1[ultimoPunto:punto] # El cromosoma del hijo2 recibe 
                                                            # parte del cromosoma del primer
                                                            # padre.
                else:
                    hijo1Cromo += cromo1[ultimoPunto:punto] # El cromosoma del hijo1 recibe 
                                                            # parte del cromosoma del primer
                                                            # padre.
                    hijo2Cromo += cromo2[ultimoPunto:punto] # El cromosoma del hijo2 recibe 
                                                            # parte del cromosoma del segundo
                                                            # padre.
                
                # Cambiamos el segmento
                switch = not switch # De esta manera garantizamos que vamos a 
                                    # ir alternando.
                ultimoPunto = punto # Nos movemos al siguiente punto de corte.
            
            # Añadimos el nuevo cromosoma a los hijos
            hijo1.append(hijo1Cromo)
            hijo2.append(hijo2Cromo)
        
        # Añadimos los dos hijos generados a la lista de hijos
        hijos.append(hijo1)
        hijos.append(hijo2)
    
    # Para este punto ya tenemos la lista de los hijos, así que lo que toca es
    # eliminar uno para hacerle espacio al individuo más apto.
    index_a_eliminar = random.randint(0, len(hijos)-1) # De manera aleatoria, 
                                                       # elegimos un índice para
                                                       # quitar un hijo de la lista.
    #print(hijos)
    #print(f'Eliminando al hijo en la posición: {index_a_eliminar}')
    del hijos[index_a_eliminar] # Borramos al hijo elegido.
    
    return hijos # Retornamos una lista de 2k-1 hijos (que era lo que queríamos)


**Creo que esta también se puede modificar:**

In [8]:
'''Función que realiza la mutación de un bit elegido al azar: Por cada hijo
   elegimos un cromosoma al azar, dentro de ese cromosoma elegimos un bit al
   azar, generamos un número aleatorio r en [0,1]. Si r<probaMutar, cambiamos 
   el valor del bit, en otro caso el bit se queda igual.
   Recibimos como parámetros: la lista de los hijos (en binario) y una 
   probabilidad de mutación. La función retorna la lista (en binario) de los hijos
   que (posiblemente) han sido mutados.'''
   

import copy

def Mutador_1_flip(hijos, probaMutar):
    
    hijosMutados = []
    #huboMutacion = False # Variable para verificar si hubo al menos una mutación.

    for hijo in hijos:
        hijo = copy.deepcopy(hijo)  # Hacemos una copia profunda del hijo para no modificar el 
                                    # original. si no hacemos una copia de los hijos antes de 
                                    # realizar mutaciones, estaremos modificando las listas 
                                    # originales directamente. Lo que puede causarnos problemas
                                    # si luego intentamos usar los hijos originales en alguna 
                                    # otra operación.          

        if isinstance(hijo[0], list):  # Si el primer elemento es una lista, tenemos listas de 
                                       # cromosomas (o sea, la dimensión del problema es al 
                                       # menos 2).
            # Elegimos un cromosoma al azar dentro del hijo

            cromElegido_index = random.randint(0, len(hijo)-1) # Elegimos un índice al azar en 
                                                           # `hijo`. Este corresponde a un 
                                                           # cromosoma.
            cromElegido = hijo[cromElegido_index] # Fijamos el cromosoma elegido.

            bit_index = random.randint(0, len(cromElegido)-1) # Elegimos un índice al azar en 
                                                              # el cromosoma elegido, este 
                                                              # corresponde a alguno de los 
                                                              # nBits.
        
            r = random.random() # Generamos un número aleatorio entre 0 y 1.

            # Si el número aleatorio es menor que la probabilidad de mutación p_m
            if r < probaMutar:
            
                # Hacemos flip al bit: si es 0 lo cambiamos a 1, si es 1 lo cambiamos a 0
                cromElegido[bit_index] = 1 if cromElegido[bit_index] == 0 else 0

                #huboMutacion = True

            # Añadimos el cromosoma (posiblemente mutado) a la lista de hijosMutados
            hijosMutados.append(hijo)

        else: # Si el  no es una lista de listas, es directamente un cromosoma (o sea, la
              # dimensión del problema es 1).
        
            bit_index = random.randint(0, len(hijo)-1) # Elegimos un bit al azar dentro del 
                                                       # cromosoma (en este caso, cada hijo
                                                       # es su propio cromosoma).
            
            r = random.random() # Generamos un número aleatorio entre 0 y 1.

            # Si el número aleatorio es menor que la probabilidad de mutación p_m
            if r < probaMutar:
                hijo[bit_index] = 1 if hijo[bit_index] == 0 else 0 # Flip del bit.

                #huboMutacion = True

            # Añadimos el cromosoma (posiblemente mutado) a la lista de hijosMutados
            hijosMutados.append(hijo)
    
    #print("Hubo mutación:", huboMutacion)

    return hijosMutados


**NOTA:** La razón por la que elegimos un hijo al azar para eliminarlo de la lista (corriendo el riesgo de perder una evaluación mejor que la que nos da el individuo previamente marcado como `masApto`) es que sabemos que el reemplazo con elitismo hace a nuestro algoritmo propenso a la convergencia prematura. Entonces, una manera de tratar de contrarrestar esto es permitir que individuos potencialmente mejores se pierdan y quedarnos con una población (posiblemente) un poco más diversa.

Funciones que sirven para decodificar un vector de $n$ arreglos de bits en vectores en $\mathbb{R}^n$ 

**Cambié esto:**

In [9]:
'''Funciones auxiliares para pasar a los hijos (posiblemente) mutados de su 
   codificación en arreglos de bits a vectores de números reales.'''

def decodifica(x_cod, nBits, intervalo):
    """
    Decodifica un vector de bits como un número real en el intervalo [a, b].

    Parámetros:
    x_cod: vector de bits a decodificar
    nBits: número de bits a utilizar
    intervalo: De la forma [a, b]

    Retorno:
    número real decodificado
    """

    a, b = intervalo
    precision = (b - a) / ((2 ** nBits) - 1)  # Calcula la precisión de la representación
    indice = int(''.join(map(str, x_cod)), 2)  # Convierte el vector de bits en un número entero
    x_dec = a + indice * precision  # Calcula el número real decodificado

    return x_dec

def decodifica_vector(vector_binario, nBits, intervalo):
    """
    Decodifica un vector de arreglos binarios en un vector de números reales.

    Parámetros:
    vector_binario: lista de listas de enteros, donde cada sub-lista representa un número codificado en binario
    nBits: número de bits a utilizar para cada número
    a: límite inferior de la partición
    b: límite superior de la partición

    Retorno:

    lista de números reales decodificados
    """

    dim_x = len(vector_binario)


    vector_reales = [decodifica(vector_binario[i], nBits, intervalo) 
                     for i in range(dim_x)]

    return vector_reales


**Cambié esto:**

In [10]:
'''Función que decodifica a los hijos mutados para el reemplazo en la 
   siguiente generación. Recibe como parámetros a los hijos (posiblemente) 
   mutados (codificados en bits), la cantidad de nBits usada en la codificación
   y el intervalo de nuestra función objetivo.   
   Retorna la lista de los hijos como vectores de números reales.'''

def Hijos_Decodificados(hijosMutados, nBits, intervalo):
    hijosDecodificados = [decodifica_vector(hijoMutado, nBits, intervalo) 
                          for hijoMutado in hijosMutados]


    return(hijosDecodificados)


In [11]:
'''Función que nos da la siguiente generación completa. Recibe como
   parámetros la lista de los 2k-1 hijos decodificados como vectores de 
   números reales y agrega a la lista al individuo más apto para 
   tener el reemplazo con elitismo.'''

def Siguiente_Gen_Elitista(hijosDecodificados, masApto):
    siguenteGeneracion = hijosDecodificados + [masApto]
    return siguenteGeneracion


Algoritmo Genético: Versión de reemplazo generacional con elitismo (sólo el mejor padre sobrevive).

In [12]:
'''Función que realiza una ejecución del algoritmo genético. Recibe como
   parámetros: la función objetivo a minimizar, el intervalo de ésta, la
   dimensión del problema, la mitad del tamaño de la población (pues vamos
   a tomar 2*(kindividuos)), la cantidad de bits para la codificación en 
   arreglos binarios, el número de cortes para el operador de reproducción 
   de cruza, la probabilidad de mutación de un bit y el número de iteraciones
   a realizar dentro de la ejecución (el número de generaciones).'''

def Algoritmo_Genetico_Elitista(funcion, intervalo, dimension,   # Cuando tienes una función que
                                kindividuos, nBits, nCortes,     # tiene muchos parámetros, la
                                probaMutar, iteraciones):        # puedes poner así para que sea
                                                                 # más legible.
    
    contadorIteraciones = 0 # Contamos las iteraciones.

    # Generamos a nuestra población inicial:
    poblacion = Generar_Poblacion(kindividuos, dimension, intervalo) 
    
    while contadorIteraciones <= iteraciones:

        # Calculamos las probabilidades de selección y obtenemos el individuo más apto
        probas_pi, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion)

        # Calculamos las probabilidades acumulativas
        probasAcumulativas_qi = Probas_Acumulativas(probas_pi)

        # Seleccionamos a los padres (ponemos `_` porque no nos interesa el número de 
        # clones).
        padres, _ = Generar_Parejas(poblacion, probasAcumulativas_qi)

        # Convertimos a los padres en arreglos de bits.
        padresBinarios = Padres_Binarios(padres, nBits, intervalo)

        # Cruzamos a los padres para generar hijos (como arreglos binarios).
        hijosBinarios = Cruzar_N_Puntos(padresBinarios, nCortes, nBits)

        # Aplicamos la mutación 1-flip a los hijos.
        hijosMutados = Mutador_1_flip(hijosBinarios, probaMutar)

        # Decodificamos a los hijos a su representación original como vectores de reales.
        hijosDecodificados = Hijos_Decodificados(hijosMutados, nBits, intervalo)

        # Generamos la siguiente generación, incluyendo el individuo más apto.
        poblacion = Siguiente_Gen_Elitista(hijosDecodificados, masApto)

        contadorIteraciones += 1 # Incrementamos en uno el contador de las iteraciones.

    return masApto # Devolvemos la mejor solución encontrada.


**Funciones de prueba:**

**Cambié esto:**

In [13]:
def Sphere(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de la esfera en el vector.
  Se usa en el intervalo de búsqueda [-5.12, 5.12] y su óptimo global está en f([0,..,0])=0

  Ejemplo de uso:
    > Sphere([0,0])

    > 0
    """
  suma = np.sum(np.array(lista) ** 2)
  return suma

intervaloSphere = (-5.12, 5.12)


def Ackley(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función Ackley en el vector.
  Se usa en el intervalo de búsqueda [-30, 30] y su óptimo global está en f([0,..,0])=0

  Ejemplo de uso:
    > Ackley([0,0])

    > 0
    """
  n = len(lista)
  lista_array = np.array(lista)  # Convertir a array de NumPy

  suma1 = np.sum(lista_array ** 2)  # Suma de los cuadrados
  suma2 = np.sum(np.cos(2 * np.pi * lista_array))  # Suma de los cosenos

  return 20 + np.exp(1) - 20 * np.exp(-0.2 * np.sqrt((1/n) * suma1)) - np.exp((1/n) * suma2)

intervaloAckley = (-30, 30)


def Griewank(x):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función Griewank en el vector.
  Se usa en el intervalo de búsqueda [-600, 600] y su óptimo global está en f([0,..,0])=0

  Ejemplo de uso:
    > Griewank([0,0])

    > 0
  """
  n = len(x)
  x_array = np.array(x, dtype=float)  # Convertir a array de NumPy

  # Calcular suma
  suma = np.sum((x_array ** 2) / 4000)

  # Calcular el producto de cosenos
  indices = np.arange(1, n + 1)  # Crear un array de índices
  cosenos = np.cos(x_array / np.sqrt(indices))  # Calcular los cosenos
  m = np.prod(cosenos)  # Calcular el producto de los cosenos

  return 1 + suma - m

intervaloGriewank = (-600, 600)


def Rastrigin(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rastrigin en el vector.
  Tiene mínimo global en f([0,..,0])

  Ejemplo de uso:
    > Rastrigin([2,3,6,7])

    > 98.0

  """
  n = len(lista)
  lista_array = np.array(lista)  # Convertir a array de NumPy

  # Calcular la suma de forma vectorizada
  suma = np.sum(lista_array ** 2 - 10 * np.cos(2 * np.pi * lista_array))

  return 10 * n + suma

intervaloRastrigin = (-5.12, 5.12)


def Rosenbrock(lista):
  """
  Esta función recibe como parámetro un vector de n entradas, ingresado como una lista y devuelve el escalar que resulta de evaluar la función de Rosenbrock en el vector.
  Tiene un mínimo global en f(1, ..., 1)

  Ejemplo de uso:
    > Rosenbrock([1,1])

    > 0

  """
  lista_array = np.array(lista)  # Convertir a array de NumPy
  n = len(lista_array)

  # Calcular la suma de forma vectorizada
  suma = np.sum(100 * (lista_array[1:n] - lista_array[0:n-1] ** 2) ** 2 + (lista_array[0:n-1] - 1) ** 2)

  return suma

intervaloRosenbrock = (-2.048, 2.048)


Veamos un ejemplo de ejecución de nuestro algoritmo para nuestras funciones de prueba:

Rosenbrock:

In [16]:
import profile

np.random.seed(1)

funcion = Rosenbrock

intervalo = intervaloRosenbrock

dimension = 10
kindividuos = 35 # Tendremos una población del doble
nBits = 15
nCortes = 12
probaMutar = 0.01
iteraciones = 1000

ejecuciones = 1
j = 0

evaluaciones = [] 

'''while j <= ejecuciones:
    masApto = Algoritmo_Genetico_Elitista(funcion, intervalo, dimension,   
                                          kindividuos, nBits, nCortes,     
                                          probaMutar, iteraciones)
    #print(f"Rosenbrock:, {str(masApto):<10}, {funcion(masApto)}")
    j += 1
    evaluaciones.append(Rosenbrock(masApto))

mejorDeLosMejores = min(evaluaciones)
peorDeLosMejores = max(evaluaciones)
promedioDeLosMejores = sum(evaluaciones) / len(evaluaciones)'''

'''print(f"Rosenbrock:, {mejorDeLosMejores:<10}, {peorDeLosMejores:<10}, {promedioDeLosMejores:<10}")'''

profile.run('Algoritmo_Genetico_Elitista(funcion, intervalo, dimension, kindividuos, nBits, nCortes, probaMutar, iteraciones)')


         97453011 function calls (85805344 primitive calls) in 112.078 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      993    2.094    0.002   23.344    0.024 1271697151.py:7(Cruzar_N_Puntos)
      993    0.359    0.000   77.812    0.078 143847220.py:12(Mutador_1_flip)
        1    0.188    0.188  112.078  112.078 156064499.py:9(Algoritmo_Genetico_Elitista)
        1    0.000    0.000    0.000    0.000 1602614505.py:8(Generar_Poblacion)
    69510    0.703    0.000    1.406    0.000 1903157480.py:86(Rosenbrock)
      992    0.047    0.000    2.625    0.003 2284899033.py:7(Hijos_Decodificados)
    68448    0.516    0.000    2.578    0.000 2324325546.py:24(decodifica_vector)
   684480    0.875    0.000    2.031    0.000 2324325546.py:4(decodifica)
      992    0.000    0.000    0.000    0.000 2774886129.py:6(Siguiente_Gen_Elitista)
    70002    1.672    0.000    2.891    0.000 32895884.py:14(Seleccionar_Padre_Ruleta)
   

KeyboardInterrupt: 